In [1]:
from utz import *
from bs4 import BeautifulSoup as bs
import plotly.graph_objects as go
import plotly.express as px

### Load one year of NJSP fatal crash data

In [2]:
def get_children(tag):
    return [ child for child in tag.children if not isinstance(child, str) ]

In [3]:
def parse_file(path):
    with open(path, 'r') as f:
        xml = bs(f)
    children = list(xml.children)
    assert len(children) == 3
    html = children[-1]
    assert html.name == 'html'
    fauqstats = html.body.fauqstats
    assert fauqstats.name == 'fauqstats'
    rundate = fauqstats.rundate.text
    year = int(fauqstats.statsyear.text)
    counties = fauqstats.find_all('county', recursive=False)
    total_accidents = int(fauqstats.totaccidents.text)
    total_injuries = int(fauqstats.totinjuries.text)
    total_fatalities = int(fauqstats.totfatalities.text)
    crash_counties = [ county for county in counties if county.municipality ]
    print(f'{len(counties)} "county" entries, {len(crash_counties)} containing "municipality"/crash info, {total_accidents} accidents, {total_injuries} injuries, {total_fatalities} fatalities')
    records = []
    for county in crash_counties:
        municipality = county.municipality
        assert municipality.name == 'municipality'
        children = get_children(municipality)
        accidents = municipality.find_all('accident', recursive=False)
        if len(children) != len(accidents):
            raise ValueError(f'Found {len(children)} municipality children, but {len(accidents)} accidents: {county}. {accidents}')
        for accident in accidents:
            obj = { child.name: child.text for child in get_children(accident) }
            obj = dict(**county.attrs, **municipality.attrs, **accident.attrs, **obj, )
            records.append(obj)
    
    df = pd.DataFrame(records)
    totals_df = pd.DataFrame([dict(
        year=year,
        accidents=total_accidents,
        injuries=total_injuries,
        fatalities=total_fatalities,
    )])
    return df, totals_df

## Load all years of NJSP data (2008-2022)

In [4]:
crashes, totals = [
    pd.concat(dfs)
    for dfs in
    zip(*[ parse_file(f'FAUQStats{year}.xml') for year in range(2008, 2023) ])
]
totals = totals.set_index('year')
totals

540 "county" entries, 519 containing "municipality"/crash info, 555 accidents, 414 injuries, 590 fatalities
528 "county" entries, 507 containing "municipality"/crash info, 550 accidents, 352 injuries, 584 fatalities
525 "county" entries, 504 containing "municipality"/crash info, 530 accidents, 366 injuries, 556 fatalities
575 "county" entries, 554 containing "municipality"/crash info, 586 accidents, 517 injuries, 627 fatalities
546 "county" entries, 525 containing "municipality"/crash info, 553 accidents, 382 injuries, 589 fatalities
498 "county" entries, 477 containing "municipality"/crash info, 508 accidents, 393 injuries, 542 fatalities
517 "county" entries, 496 containing "municipality"/crash info, 523 accidents, 345 injuries, 556 fatalities
517 "county" entries, 496 containing "municipality"/crash info, 522 accidents, 374 injuries, 562 fatalities
550 "county" entries, 529 containing "municipality"/crash info, 570 accidents, 398 injuries, 602 fatalities
580 "county" entries, 559 co

,accidents,injuries,fatalities
year,,,
2008,555,414,590
2009,550,352,584
2010,530,366,556
2011,586,517,627
2012,553,382,589
2013,508,393,542
2014,523,345,556
2015,522,374,562
2016,570,398,602


In [5]:
crashes['dt'] = crashes[['date','time']].apply(lambda r: to_dt(f'{r["date"]} {r["time"]}'), axis=1)
crashes = (
    crashes
    .astype({
        'fatalities': float,
        'fatal_d': float,
        'fatal_p': float,
        'fatal_t': float,
        'fatal_b': float,
        'injuries': float,
    })
    .drop(columns=['date', 'time'])
    .set_index('accid')
)
crashes

,ccode,cname,mcode,mname,highway,location,fatalities,injuries,street,fatal_d,fatal_p,fatal_t,fatal_b,dt
accid,,,,,,,,,,,,,,
2391,15,Ocean,1512,Lacey Twsp,444,State/Interstate Authority 444 S MP 72.4,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2008-12-31 13:40:00
2390,15,Ocean,1512,Lacey Twsp,614,County 614 E MP 12.5,1.0,0.0,NaN,NaN,NaN,NaN,NaN,2008-12-29 18:39:00
2388,18,Somerset,1814,North Plainfield Bo,22,State Highway 22 E MP 45.25 at North Drive,1.0,0.0,NaN,NaN,NaN,NaN,NaN,2008-12-29 06:10:00
2389,03,Burlington,0325,Mount Laurel Twsp,537,County 537 E MP 12.74,1.0,0.0,NaN,NaN,NaN,NaN,NaN,2008-12-29 03:37:00
2384,13,Monmouth,1319,Howell Twsp,9,State Highway 9 S MP 105.08 at Friendship Road,1.0,0.0,NaN,NaN,NaN,NaN,NaN,2008-12-26 17:52:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11465,20,Union,2004,Elizabeth City,1,State Highway 1,1.0,NaN,NaN,1.0,0.0,0.0,0.0,2022-01-02 11:26:00
11462,08,Gloucester,0805,Franklin Twsp,55,State Highway 55 MP 41.2,1.0,0.0,NaN,1.0,0.0,0.0,0.0,2022-01-02 07:54:00
11461,12,Middlesex,1202,Cranbury Twsp,615,County 615,1.0,NaN,NaN,0.0,0.0,1.0,0.0,2022-01-02 05:43:00


#### Save to file

In [6]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///njsp.db")

tables = {
    'totals': totals,
    'crashes': crashes,
}

for name, table in tables.items():
    table.to_sql(name, con=engine, if_exists='replace',)

#### Group by year

In [8]:
dt = crashes.dt.dt
fatalities_per_year = crashes.fatalities.groupby(dt.year).sum().astype(int)
sxs(
    fatalities_per_year,
    totals.fatalities.rename('total fatalities'),
    (totals.fatalities - fatalities_per_year).rename('diff'),
)

,fatalities,total fatalities,diff
2008,558,590,32
2009,546,584,38
2010,533,556,23
2011,594,627,33
2012,559,589,30
2013,513,542,29
2014,532,556,24
2015,533,562,29
2016,562,602,40
2017,595,624,29


#### Group by month

In [10]:
ym = crashes.dt.apply(lambda d: d.strftime('%Y-%m'))
fatalities_per_month = crashes.fatalities.groupby(ym).sum()
fatalities_per_month = fatalities_per_month.loc[fatalities_per_month.index < '2022-04']
fatalities_per_month

dt
2008-01    54.0
2008-02    40.0
2008-03    32.0
2008-04    44.0
2008-05    44.0
           ... 
2021-11    70.0
2021-12    61.0
2022-01    54.0
2022-02    40.0
2022-03    42.0
Name: fatalities, Length: 171, dtype: float64

#### Rolling avg

In [11]:
rolling = fatalities_per_month.rolling(12).mean()
rolling

dt
2008-01          NaN
2008-02          NaN
2008-03          NaN
2008-04          NaN
2008-05          NaN
             ...    
2021-11    53.583333
2021-12    55.000000
2022-01    55.750000
2022-02    56.916667
2022-03    56.000000
Name: fatalities, Length: 171, dtype: float64

#### Fatalities per month plot

In [13]:
margin = 40

fig = go.Figure()
fig.add_trace(go.Bar(x=fatalities_per_month.index, y=fatalities_per_month.values, name='Fatalities'))
fig.add_trace(go.Scatter(x=rolling.index, y=rolling.values, name='12mo avg', line={'width': 3, 'color': 'black', }))
fig.update_layout(
    title='NJ Traffic Fatalities per Month',
    title_x=0.5,
    margin=dict(l=margin, r=margin, t=margin, b=margin),
)
fig.write_image('fatalities_per_month.png', width=1200, height=600)
fig.show()

#### Fatalities per year plot

In [14]:
fig = px.bar(
    totals.fatalities[totals.index < 2022],
    labels={
        'variable': '',
        'dt': 'Year',
        'value': 'Fatalities'
    },
)
fig.update_layout(
    title='NJ Traffic Fatalities per Year',
    title_x=0.5,
    showlegend=False,
)
fig.write_image('fatalities_per_year.png')
fig

In [16]:
mos = (
    sxs(
        dt.year.rename('year'),
        dt.month.rename('month'),
        crashes.fatalities,
    )
    .groupby(['year', 'month']).sum()
    .iloc[:-1]
)
mos

fatalities
year month            
2008 1            54.0
     2            40.0
     3            32.0
     4            44.0
     5            44.0
...                ...
2021 11           70.0
     12           61.0
2022 1            54.0
     2            40.0
     3            42.0

[171 rows x 1 columns]

In [17]:
@dataclass
class RGB:
    r: float
    g: float
    b: float

    @staticmethod
    def from_css(css):
        if css[0] == '#':
            css = css[1:]
        
        m = fullmatch('(?P<r>[\da-f]{2})(?P<g>[\da-f]{2})(?P<b>[\da-f]{2})', css)
        if m:
            pcs = [ m['r'], m['g'], m['b'] ]
        else:
            m = fullmatch('(?P<r>[\da-f])(?P<g>[\da-f])(?P<b>[\da-f])', css)
            assert m
            pcs = [ m['r'] * 2, m['g'] * 2, m['b'] * 2 ]
            
        r, g, b = [ int(s, 16) for s in pcs ]
        return RGB(r, g, b)

    def __add__(l, r):
        return RGB(l.r + r.r, l.g + r.g, l.b + r.b)
    
    def __sub__(l, r):
        return RGB(l.r - r.r, l.g - r.g, l.b - r.b)
    
    def __mul__(self, f):
        return RGB(self.r * f, self.g * f, self.b * f)
    
    def __truediv__(self, f):
        return RGB(self.r / f, self.g / f, self.b / f)    
    
    @property
    def css(s):
        return '#%s' % ''.join([ '%02x' % int(f) for f in [ s.r, s.g, s.b ]])

In [18]:
pivoted = mos.reset_index().sort_values(['month', 'year'])

In [19]:
min_color = RGB.from_css('#ddd')
max_color = RGB.from_css('#000')
years = pivoted.year.unique()
N = len(years)
delta = (max_color - min_color) / N
colors = [ min_color + delta*idx for idx in range(N+1) ]
colors

[RGB(r=221.0, g=221.0, b=221.0),
 RGB(r=206.26666666666668, g=206.26666666666668, b=206.26666666666668),
 RGB(r=191.53333333333333, g=191.53333333333333, b=191.53333333333333),
 RGB(r=176.8, g=176.8, b=176.8),
 RGB(r=162.06666666666666, g=162.06666666666666, b=162.06666666666666),
 RGB(r=147.33333333333334, g=147.33333333333334, b=147.33333333333334),
 RGB(r=132.60000000000002, g=132.60000000000002, b=132.60000000000002),
 RGB(r=117.86666666666667, g=117.86666666666667, b=117.86666666666667),
 RGB(r=103.13333333333334, g=103.13333333333334, b=103.13333333333334),
 RGB(r=88.4, g=88.4, b=88.4),
 RGB(r=73.66666666666669, g=73.66666666666669, b=73.66666666666669),
 RGB(r=58.93333333333334, g=58.93333333333334, b=58.93333333333334),
 RGB(r=44.20000000000002, g=44.20000000000002, b=44.20000000000002),
 RGB(r=29.46666666666667, g=29.46666666666667, b=29.46666666666667),
 RGB(r=14.733333333333348, g=14.733333333333348, b=14.733333333333348),
 RGB(r=0.0, g=0.0, b=0.0)]

In [20]:
css_colors = [ color.css for color in colors ]
css_colors

['#dddddd',
 '#cecece',
 '#bfbfbf',
 '#b0b0b0',
 '#a2a2a2',
 '#939393',
 '#848484',
 '#757575',
 '#676767',
 '#585858',
 '#494949',
 '#3a3a3a',
 '#2c2c2c',
 '#1d1d1d',
 '#0e0e0e',
 '#000000']

In [21]:
month_names = [ to_dt('2022-%02d' % i).strftime('%b') for i in range(1, 13) ]
month_names

['Jan',
 'Feb',
 'Mar',
 'Apr',
 'May',
 'Jun',
 'Jul',
 'Aug',
 'Sep',
 'Oct',
 'Nov',
 'Dec']

In [22]:
def color_interp(l_idx, l_n, r_colors):
    r_n = len(r_colors) - 1
    r_pos = l_idx / l_n * r_n
    r_idx = int(r_pos)
    r_rem = r_pos - r_idx
    r_colors = [ RGB.from_css(r_color) for r_color in r_colors ]
    cur = r_colors[r_idx]
    if r_rem == 0:
        return cur.css
    nxt = r_colors[r_idx + 1]
    color = cur + (nxt - cur) * r_rem
    return color.css

In [23]:
def colors_lengthen(colors, n):
    return [ color_interp(i, n - 1, colors) for i in range(n) ]

In [26]:
px_colors = px.colors.sequential.Inferno
px_colors, len(colors)

(['#000004',
  '#1b0c41',
  '#4a0c6b',
  '#781c6d',
  '#a52c60',
  '#cf4446',
  '#ed6925',
  '#fb9b06',
  '#f7d13d',
  '#fcffa4'],
 10)

In [27]:
colors = list(reversed(colors_lengthen(px_colors, len(years))))
colors

['#fcffa4',
 '#f8e161',
 '#f8c12d',
 '#fa9e09',
 '#f37e17',
 '#e6612c',
 '#d34941',
 '#ba3853',
 '#9e2961',
 '#811f6a',
 '#64156c',
 '#460c68',
 '#280c4d',
 '#11072b',
 '#000004']

In [29]:
fig = px.bar(
    x = pivoted.month,
    y = pivoted.fatalities,
    color = pivoted.year.astype(str),
    color_discrete_sequence=colors,
    labels=dict(color='', x='Month', y='Traffic deaths',),
    barmode='group',
)
fig = fig.update_layout(
    paper_bgcolor='white',
    plot_bgcolor='white',
    title='NJ traffic fatalities by month',
    title_x=0.5,
    xaxis=dict(
        tickmode = 'array',
        tickvals = list(range(1, 13)),
        ticktext = month_names,
    )
)
fig.write_image('fatalities_by_month_bars.png', width=1200, height=700,)
fig

In [30]:
fig = px.line(
    x = pivoted.month,
    y = pivoted.fatalities,
    color = pivoted.year,
    color_discrete_sequence=colors,
    labels=dict(color='', x='Month', y='Traffic deaths',),
    #barmode='group',
)
fig = fig.update_layout(
    paper_bgcolor='white',
    plot_bgcolor='white',
    title='NJ traffic fatalities by month',
    title_x=0.5,
    xaxis=dict(
        tickmode = 'array',
        tickvals = list(range(1, 13)),
        ticktext = month_names,
    )
)
fig.write_image('fatalities_by_month_lines.png', width=1200, height=700,)
fig